In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import BertTokenizer

# loading the sentiment analysis model
sentiment_label_map={'positive': 0, 'negative': 1, 'neutral': 2}
sentiment_model=TFAutoModelForSequenceClassification.from_pretrained("sentiment_analysis/sentiment_results", num_labels=len(sentiment_label_map),from_pt=True)
sentiment_tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

#loading the categorization model
categorization_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
categorization_label_map={'quality': 0, 'overall': 1, 'service': 2, 'price': 3}
categorization_model=TFAutoModelForSequenceClassification.from_pretrained("results", num_labels=len(categorization_label_map),from_pt=True)

#loading the sarcasm detecion model
sarcasm_tokenizer = AutoTokenizer.from_pretrained("nikesh66/Sarcasm-Detection-using-BERT")
sarcasm_label_map={'sarcasm': 0, 'not sarcasm': 1}
sarcasm_model = TFAutoModelForSequenceClassification.from_pretrained("sarcasm_detection\sarcasm_results", num_labels=len(sarcasm_label_map),from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 mode

In [3]:
def sentiment_analysis(review,sentiment_tokenizer,sentiment_model,sentiment_label_map):
    # Tokenize the user review (assuming you have a tokenizer)
    tokenized_user_review = sentiment_tokenizer(review, truncation=True, padding=True, return_tensors="tf")

    # Make predictions
    predictions = sentiment_model.predict(dict(tokenized_user_review))

    # Extract the predicted probabilities for each class
    predicted_probabilities = predictions[0][0]

    # Get the predicted class index
    predicted_class_index = predicted_probabilities.argmax()

    def print_key_by_value(dict, value1):
      for key, value in dict.items():
        if value == value1:
          return key

    # Example usage:
    predicted_category=print_key_by_value(sentiment_label_map,predicted_class_index )
    # Print the predicted class index
    return predicted_category

In [4]:
def categorize(review,categorization_tokenizer,categorization_model,categorization_label_map):
# Tokenize the user review (assuming you have a tokenizer)
    tokenized_user_review = categorization_tokenizer(review, truncation=True, padding=True, return_tensors="tf")

# Make predictions
    predictions = categorization_model.predict(dict(tokenized_user_review))

# Extract the predicted probabilities for each class
    predicted_probabilities = predictions[0][0]

# Get the predicted class index
    predicted_class_index = predicted_probabilities.argmax()

    def print_key_by_value(dict, value1):
      for key, value in dict.items():
        if value == value1:
          return key

# Example usage:
    predicted_category=print_key_by_value(categorization_label_map,predicted_class_index )
# Print the predicted class index
    #print('predicted index',predicted_class_index)
    return predicted_category

In [5]:
def sarcasm_detection(review,sarcasm_tokenizer,sarcasm_model,sarcasm_label_map):
    # Tokenize the user review (assuming you have a tokenizer)
    tokenized_user_review = sarcasm_tokenizer(review, truncation=True, padding=True, return_tensors="tf")

    # Make predictions
    predictions = sarcasm_model.predict(dict(tokenized_user_review))

    # Extract the predicted probabilities for each class
    predicted_probabilities = predictions[0][0]

    # Get the predicted class index
    predicted_class_index = predicted_probabilities.argmax()

    def print_key_by_value(dict, value1):
      for key, value in dict.items():
        if value == value1:
          return key

    # Example usage:
    predicted_category=print_key_by_value(sarcasm_label_map,predicted_class_index )
    # Print the predicted class index
    return predicted_category

In [ ]:
input_text = input('enter review:')
print(sentiment_analysis(input_text,sentiment_tokenizer,sentiment_model,sentiment_label_map))
print(categorize(input_text,categorization_tokenizer,categorization_model,categorization_label_map))
print(sarcasm_detection(input_text,sarcasm_tokenizer,sarcasm_model,sarcasm_label_map))

In [7]:
import gradio as gr

def sentiment_and_category(review):
    sentiment = sentiment_analysis(review,sentiment_tokenizer,sentiment_model,sentiment_label_map)
    category = categorize(review,categorization_tokenizer,categorization_model,categorization_label_map)
    sarcasm = sarcasm_detection(review,sarcasm_tokenizer,sarcasm_model,sarcasm_label_map)
    if sarcasm=='sarcasm':
        sentiment1='negative'
    else:
        sentiment1=sentiment
    return sentiment1, category


iface = gr.Interface(fn=sentiment_and_category, 
                     inputs="text", 
                     outputs=[gr.Textbox(label="Sentiment", lines=3),gr.Textbox(label="Category", lines=3)], 
                     title="Product Review Categorization and Sentiment Analysis",
                     description="Enter your product review below and get sentiment and category analysis.")

iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 1s 700ms/step
